# Milestone 2 - Relation Classification Neural Network Model

In [834]:
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn, matmul
from torch.nn.functional import softmax
from torchmetrics import Accuracy


import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

## Load dataset

In [835]:
import pandas as pd

# Load FoodDisease dataset preprocessed in Milestone 1
feature_cols = ['food_entity', 'disease_entity', 'sentence', 'is_cause', 'is_treat', 'tokens', 'tokens_stem', 'tokens_lemma', 'sdp_tokens_lemma']

df = pd.read_csv(
    "../data/processed/df.csv",
    sep=',', quotechar='"',
    skipinitialspace=True,
    encoding='utf-8',
    on_bad_lines='skip',
    usecols=feature_cols)

feature_cols = ['food_entity', 'disease_entity', 'sentence', 'tokens', 'tokens_stem', 'tokens_lemma', 'sdp_tokens_lemma']
label_cols = ['is_cause', 'is_treat']

y = df[label_cols].to_numpy()


## Tokenization

In [836]:
from keras.preprocessing.text import one_hot
from nltk.tokenize import word_tokenize
from keras_preprocessing.sequence import pad_sequences
import copy

df_x = df["sdp_tokens_lemma"]

X_tmp = copy.deepcopy(df_x)
embedded_sentences = [one_hot(sent, 10000) for sent in X_tmp]
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(df_x, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))
print(length_long_sentence)
# will put length of longest sentence +1 bc 31 is not divisible by anything,
# this is important for the number of heads in the transformer
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence+1, padding='post')


31


In [837]:
print(padded_sentences)

[[5645 4369 7744 ...    0    0    0]
 [5645 4369 5907 ...    0    0    0]
 [5645 4369  695 ...    0    0    0]
 ...
 [5645 4369 3523 ...    0    0    0]
 [5645 4369 7324 ...    0    0    0]
 [5645 4369 7420 ...    0    0    0]]


## Dataset split

In [838]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_sentences,y,  test_size=0.1, random_state=1)
X_train, X_val, y_train, y_val= train_test_split(X_train, y_train, test_size=0.111, random_state=1) # 0.111 x 0.9 = 0.1

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(475, 32)
(60, 32)
(60, 32)


In [839]:
class TorchDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y

    def get_x(self):
        return self.x

    def get_y(self):
        return self.y
            
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return (torch.tensor(x).float(), torch.tensor(y).long())
    
    def __len__(self):
        return self.x.shape[0]

    def get_dataloader(self, batch_size=128, num_workers=0, shuffle=False):
        return DataLoader(self, batch_size=batch_size, drop_last=True, pin_memory=True, num_workers=num_workers, shuffle=shuffle)



## Data Loaders

In [840]:
X_train = TorchDataset(X_train,y_train)
X_test = TorchDataset(X_test, y_test)
X_val = TorchDataset(X_val, y_val)

dataloaders = { 'train': X_train.get_dataloader(batch_size=length_long_sentence+1, shuffle=True), 'test': X_test.get_dataloader(batch_size=length_long_sentence+1, shuffle=False), 'val': X_val.get_dataloader(batch_size=length_long_sentence+1, shuffle=False) }

# Multi Head Attention Model

In [841]:
def scaled_dot_product(query, key, value, mask=None):
        dk = query.size()[-1]
        scores = query.matmul(key.transpose(-2, -1)) / math.sqrt(dk)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attention = F.softmax(scores, dim=-1)
        values = torch.matmul(attention, value)
        return values

In [842]:
class MultiheadAttention(nn.Module):
    
    def __init__(self, input_dim, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0, "Embedding dimension must be 0 modulo number of heads."

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        self.qkv_proj = nn.Linear(input_dim, 3*embed_dim)
        self.o_proj = nn.Linear(embed_dim, embed_dim)

    def forward(self, x, mask=None):
        batch_size, seq_length, _ = x.size()
        qkv = self.qkv_proj(x)

        # Separate Q, K, V from linear output
        qkv = qkv.reshape(batch_size, seq_length, self.num_heads, 3*self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3) # [Batch, Head, SeqLen, Dims]
        q, k, v = qkv.chunk(3, dim=-1)

        # Determine value outputs
        values = scaled_dot_product(q, k, v, mask=mask)
        values = values.permute(0, 2, 1, 3) # [Batch, SeqLen, Head, Dims]
        values = values.reshape(batch_size, seq_length, self.embed_dim)
        output = self.o_proj(values)

        return output

In [843]:
class EncoderBlock(nn.Module):
    
    def __init__(self, input_dim, num_heads, dim_feedforward, dropout=0.0):
        """
        Inputs:
            input_dim - Dimensionality of the input
            num_heads - Number of heads to use in the attention block
            dim_feedforward - Dimensionality of the hidden layer in the MLP
            dropout - Dropout probability to use in the dropout layers
        """
        super().__init__()

        # Attention layer
        self.self_attn = MultiheadAttention(input_dim, input_dim, num_heads)

        # Two-layer MLP
        self.linear_net = nn.Sequential(
            nn.Linear(input_dim, dim_feedforward),
            nn.Dropout(dropout),
            nn.ReLU(inplace=True),
            nn.Linear(dim_feedforward, input_dim)
        )

        # Layers to apply in between the main layers
        self.norm1 = nn.LayerNorm(input_dim)
        self.norm2 = nn.LayerNorm(input_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # Attention part
        attn_out = self.self_attn(x, mask=mask)
        x = x + self.dropout(attn_out)
        x = self.norm1(x)

        # MLP part
        linear_out = self.linear_net(x)
        x = x + self.dropout(linear_out)
        x = self.norm2(x)

        return x

In [844]:
class TransformerEncoder(nn.Module):
    
    def __init__(self, num_layers, **block_args):
        super().__init__()
        self.layers = nn.ModuleList([EncoderBlock(**block_args) for _ in range(num_layers)])

    def forward(self, x, mask=None):
        for l in self.layers:
            x = l(x, mask=mask)
        return x

    def get_attention_maps(self, x, mask=None):
        attention_maps = []
        for l in self.layers:
            _, attn_map = l.self_attn(x, mask=mask, return_attention=True)
            attention_maps.append(attn_map)
            x = l(x)
        return attention_maps

In [845]:
# This is very unnecessary but its here, taken from https://pytorch.org/tutorials/beginner/transformer_tutorial.html#define-the-model
class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model, max_len=5000):
        """
        Inputs
            d_model - Hidden dimensionality of the input.
            max_len - Maximum length of a sequence to expect.
        """
        super().__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)

        self.register_buffer('pe', pe, persistent=False)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

In [846]:
class TransformerPredictor(LightningModule):
    
    def __init__(self, input_dim, model_dim, num_classes, num_heads, num_layers, lr=1e-4, dropout=0.0, input_dropout=0.0, weight_decay=1e-2):
        """
        Inputs:
            input_dim - Hidden dimensionality of the input
            model_dim - Hidden dimensionality to use inside the Transformer
            num_classes - Number of classes to predict per sequence element
            num_heads - Number of heads to use in the Multi-Head Attention blocks
            num_layers - Number of encoder blocks to use.
            lr - Learning rate in the optimizer
            warmup - Number of warmup steps. Usually between 50 and 500
            max_iters - Number of maximum iterations the model is trained for. This is needed for the CosineWarmup scheduler
            dropout - Dropout to apply inside the model
            input_dropout - Dropout to apply on the input features
        """
        super().__init__()
        self.save_hyperparameters()
        self._create_model()

    def _create_model(self):
        # Input dim -> Model dim
        self.input_net = nn.Sequential(
            nn.Dropout(self.hparams.input_dropout),
            nn.Linear(self.hparams.input_dim, self.hparams.model_dim)
        )
        # Positional encoding for sequences
        self.positional_encoding = PositionalEncoding(d_model=self.hparams.model_dim)
        # Transformer
        self.transformer = TransformerEncoder(num_layers=self.hparams.num_layers,
                                              input_dim=self.hparams.model_dim,
                                              dim_feedforward=2*self.hparams.model_dim,
                                              num_heads=self.hparams.num_heads,
                                              dropout=self.hparams.dropout)
        # Output classifier per sequence lement
        self.output_net = nn.Sequential(
            nn.Linear(self.hparams.model_dim, self.hparams.model_dim),
            nn.LayerNorm(self.hparams.model_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(self.hparams.dropout),
            nn.Linear(self.hparams.model_dim, self.hparams.num_classes)
        )


    def forward(self, x, mask=None):
        """
        Inputs:
            x - Input features of shape [Batch, SeqLen, input_dim]
            mask - Mask to apply on the attention outputs (optional)
        """

        x = self.input_net(x)
        x = self.positional_encoding(x)
        x = self.transformer(x, mask=mask)
        x = self.output_net(x)
        return x

    @torch.no_grad()
    def get_attention_maps(self, x, mask=None, add_positional_encoding=True):
        """
        Function for extracting the attention matrices of the whole Transformer for a single batch.
        Input arguments same as the forward pass.
        """
        x = self.input_net(x)
        if add_positional_encoding:
            x = self.positional_encoding(x)
        attention_maps = self.transformer.get_attention_maps(x, mask=mask)
        return attention_maps

    def configure_optimizers(self):
            return torch.optim.Adam(
            self.parameters(), lr=self.hparams.lr,
            weight_decay=self.hparams.weight_decay)

    def _loss_fn(self, out, y):
        y = y.unsqueeze(0)
        out = out.float()
        y = y.float()
        loss = nn.BCEWithLogitsLoss()(out, y) # Multiclass classification
        return loss
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self(x)
        loss = self._loss_fn(out, y)
        self.log('loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        out = self(x)
        loss = self._loss_fn(out, y)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        accuracy = Accuracy()
        acc = accuracy(out[0], y)
        self.log('accuracy', acc, on_epoch=True, prog_bar=True, logger=True)
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        out = self(x)
        loss = self._loss_fn(out, y)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        accuracy = Accuracy()
        acc = accuracy(out[0], y)
        self.log('accuracy', acc, on_epoch=True, prog_bar=True, logger=True)
    
    

In [847]:
class TorchTrainer():
    def __init__(self, model, name, dirpath, dataloaders, max_epochs=50) -> None:
        self.model = model
        self.name = name
        self.dirpath = dirpath
        self.max_epochs = max_epochs
        self.dataloaders = dataloaders

    def run(self):
        logger = TensorBoardLogger(f"{self.dirpath}/tensorboard", name=self.name)
        callbacks = [
            ModelCheckpoint(dirpath=Path(self.dirpath, self.name), monitor="val_loss"),
            EarlyStopping(monitor='loss')]
        trainer = Trainer(deterministic=True, logger=logger, callbacks=callbacks, max_epochs=self.max_epochs)
        trainer.fit(self.model, self.dataloaders['train'], self.dataloaders['val'])
        val_result = trainer.test(self.model, self.dataloaders['val'], verbose=True)
        test_result = trainer.test(self.model, self.dataloaders['test'], verbose=True)
        result = {"test_acc": test_result, "val_acc": val_result}
        return result

In [848]:
model = TransformerPredictor(input_dim=length_long_sentence+1, model_dim=length_long_sentence+1, num_layers=2, num_heads=8, num_classes=2)

trainer = TorchTrainer(model, 'test', './', dataloaders, max_epochs=100)

c:\Users\Ana\miniconda3\envs\tuwnlpie\lib\site-packages\pytorch_lightning\core\lightning.py:22: LightningDeprecationWarning: pytorch_lightning.core.lightning.LightningModule has been deprecated in v1.7 and will be removed in v1.9. Use the equivalent class from the pytorch_lightning.core.module.LightningModule class instead.
  rank_zero_deprecation(


In [849]:
result = trainer.run()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Ana\miniconda3\envs\tuwnlpie\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:604: UserWarning: Checkpoint directory C:\Users\Ana\Desktop\FER\dip\tu_wien\nlp\project-1div7\docs\test exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name                | Type               | Params
-----------------------------------------------------------
0 | input_net           | Sequential         | 1.1 K 
1 | positional_encoding | PositionalEncoding | 0     
2 | transformer         | TransformerEncoder | 17.1 K
3 | output_net          | Sequential         | 1.2 K 
-----------------------------------------------------------
19.3 K    Trainable params
0         Non-trainable params
19.3 K    Total params
0.077     Total estimated model params size (MB)


c:\Users\Ana\miniconda3\envs\tuwnlpie\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\Ana\miniconda3\envs\tuwnlpie\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\Ana\miniconda3\envs\tuwnlpie\lib\site-packages\pytorch_lightning\trainer\trainer.py:1558: PossibleUserWarning: The number of training batches (14) is smaller than the logging interv

Epoch 24: 100%|██████████| 15/15 [00:00<00:00, 26.05it/s, loss=0.605, v_num=77, loss_step=0.632, val_loss=0.612, accuracy=0.625, loss_epoch=0.607]


c:\Users\Ana\miniconda3\envs\tuwnlpie\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 22.73it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy                   0.625
        val_loss            0.6115018129348755
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 32.26it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy                 0.609375
        val_loss            0.61590409278

In [850]:
print(result['val_acc'])
print(result['test_acc'])

[{'val_loss': 0.6115018129348755, 'accuracy': 0.625}]
[{'val_loss': 0.6159040927886963, 'accuracy': 0.609375}]
